In [1]:
import pickle
import time
import math
import random

class HammingSA:
    def __init__(self, nam_file):
        with open(nam_file, 'rb') as f:
            self.codes_int = pickle.load(f)
        self.num_nodes = len(self.codes_int) - 1

    def are_neighbors(self, u: int, v: int) -> bool:
        if u == v: return False
        if u > v: u, v = v, u # Ensure upper triangular consistency
        return ((self.codes_int[u] >> v) & 1) == 0

    def get_conflicts(self, S):
        count = 0
        for i in range(len(S)):
            for j in range(i + 1, len(S)):
                if not self.are_neighbors(S[i], S[j]):
                    count += 1
        return count

    def solve(self, v_self, target=16, max_iter=20000):
        # 1. Initialization: Start with a smaller random subset of neighbors
        neighbors = [i for i in range(1, self.num_nodes + 1) if i != v_self and self.are_neighbors(v_self, i)]
        random.shuffle(neighbors)
        S = [v_self] + neighbors[:10] 
        
        current_conflicts = self.get_conflicts(S)
        
        # 2. SA Parameters
        T = 5.0            # Initial Temperature
        T_min = 0.0001
        alpha = 0.9997     # Very slow cooling to navigate Hamming symmetries
        
        best_S = list(S)
        best_size = 0

        print(f"Annealing for Agent {v_self} on Hamming 8-4...")

        for i in range(max_iter):
            if T <= T_min: break
            
            # 3. Propose a move
            new_S = list(S)
            rand_val = random.random()
            
            # Add, Remove, or Swap (Swap helps escape local minima faster)
            if rand_val < 0.3 and len(new_S) > 1:
                # Remove
                node_to_remove = random.choice([u for u in new_S if u != v_self])
                new_S.remove(node_to_remove)
            elif rand_val < 0.6:
                # Add
                potential = [n for n in range(1, self.num_nodes + 1) if n not in new_S]
                if potential:
                    new_S.append(random.choice(potential))
            else:
                # Swap one non-v_self node for a new one
                if len(new_S) > 1:
                    node_to_remove = random.choice([u for u in new_S if u != v_self])
                    new_S.remove(node_to_remove)
                    potential = [n for n in range(1, self.num_nodes + 1) if n not in new_S]
                    if potential:
                        new_S.append(random.choice(potential))

            # 4. Energy Calculation
            new_conflicts = self.get_conflicts(new_S)
            
            # Energy function: Penalize conflicts heavily, reward size
            # For Hamming, conflicts are very 'expensive'
            delta_energy = (new_conflicts - current_conflicts) * 2.0 - (len(new_S) - len(S))

            # 5. Acceptance Criteria
            if delta_energy < 0 or random.random() < math.exp(-delta_energy / T):
                S = new_S
                current_conflicts = new_conflicts
            
            # 6. Update Best Valid Clique
            if current_conflicts == 0:
                if len(S) > best_size:
                    best_size = len(S)
                    best_S = list(S)
                    print(f"Iteration {i}: Found Valid Clique Size {best_size} (T={T:.4f})")
                    
                if best_size >= target:
                    print("TARGET SIZE 16 REACHED!")
                    break

            T *= alpha 
            
        return best_S, best_size

# --- Execution ---
solver = HammingSA('hamming8-4_nam.data')
start_t = time.time()
# Standard Hamming 8-4 (n=256) target is 16
final_clique, size = solver.solve(v_self=random.randint(1, 256), target=16)
end_t = time.time()

print("-" * 40)
print(f"Final Results for Hamming 8-4:")
print(f"Max Clique Size: {size}")
print(f"Execution Time: {end_t - start_t:.2f}s")
print(f"Clique: {sorted(final_clique)}")
print("-" * 40)

Annealing for Agent 32 on Hamming 8-4...
Iteration 34: Found Valid Clique Size 4 (T=4.9493)
Iteration 84: Found Valid Clique Size 5 (T=4.8756)
Iteration 1337: Found Valid Clique Size 6 (T=3.3477)
Iteration 8040: Found Valid Clique Size 7 (T=0.4480)
Iteration 8673: Found Valid Clique Size 8 (T=0.3705)
Iteration 8855: Found Valid Clique Size 9 (T=0.3508)
Iteration 8912: Found Valid Clique Size 10 (T=0.3449)
Iteration 11948: Found Valid Clique Size 11 (T=0.1387)
Iteration 11956: Found Valid Clique Size 12 (T=0.1384)
Iteration 15024: Found Valid Clique Size 13 (T=0.0551)
Iteration 16213: Found Valid Clique Size 14 (T=0.0386)
Iteration 16509: Found Valid Clique Size 15 (T=0.0353)
Iteration 17984: Found Valid Clique Size 16 (T=0.0227)
TARGET SIZE 16 REACHED!
----------------------------------------
Final Results for Hamming 8-4:
Max Clique Size: 16
Execution Time: 1.39s
Clique: [9, 32, 36, 53, 71, 82, 110, 123, 134, 147, 175, 186, 204, 221, 225, 248]
----------------------------------------
